# Explore and Process
In this notebook we explore the dataset and we process the data, making different tasks:
* Tokenization
* Delete the Stop Words
* Remove Punctuation
* Lemmatization

## Import Libraries
We used various libraries: pandas to read the dataset, pandarallel for parallel processing tasks on the dataset, nltk for removing stop words, tokenization and lemmatization and the regex library to remove the punctuation.


In [ ]:
import pandas as pd
from pandarallel import pandarallel
import ast
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

## Explore the dataset

### Read the datasets

In [ ]:
dataRew=pd.read_json('../Dataset/IMDB_reviews.json',lines=True)
dataMovie=pd.read_json('../Dataset/IMDB_movie_details.json',lines=True)

In [ ]:
dataRew.info()

There are no null fields

In [ ]:
dataRew[dataRew['review_text']=='']

In [ ]:
dataMovie.info()

### Some plot synopsis are not present
We exclude films where the synopsis is not present, as the plot summary does not contain plot twists usefull for detect spoiler.

In [ ]:
dataMovie[dataMovie["plot_synopsis"]=='']

In [ ]:
dataMovie=dataMovie[dataMovie["plot_synopsis"]!='']

In [ ]:
dataMovie[dataMovie["plot_synopsis"]=='']

In [ ]:
dataMovie.head()

In [ ]:
dataRew.head()

### Compute the average,min,max length of reviews and the plot synopsis

In [ ]:
ColonnaL=dataRew["review_text"].apply(len)

In [ ]:
ColonnaL.mean()

The average length of the reviews is 1460

In [ ]:
ColonnaL.min()

Minimum review length: 18 

In [ ]:
ColonnaL.max()

Maximum review length: 14963

### Plot synopsis

In [ ]:
tempMovie=dataMovie["plot_synopsis"].apply(len)

In [ ]:
tempMovie.mean()

The average length of the plot synopsis is 9644

In [ ]:
tempMovie.min()

Minimum plot synopsis length: 45 

In [ ]:
tempMovie.max()

Maximum plot synopsis length: 63904

### Compute the min and max data of the review

In [ ]:
unique_data=dataRew["review_date"].unique()

In [ ]:
data = pd.to_datetime(unique_data)

In [ ]:
unique_data

In [ ]:
data

In [ ]:
data.min()

In [ ]:
data.max()

### Distribution of spoilers
The dataset is inbalanced

In [ ]:
values=dataRew["is_spoiler"].value_counts()

In [ ]:
values.plot(kind='bar',color=['blue','red'])
plt.xlabel('Spoiler')
plt.ylabel('Numbers reviews')
plt.title('Distribution of Spoiler in reviews')

### Estimation of pre-announced spoilers (reviews containing spoiler alert)

In [ ]:
only_text = dataRew[['movie_id', 'review_text','is_spoiler']]

In [ ]:
#Estimation: select reviews with "spoiler" word and exclude negations like "spoiler free" "not a spoiler" ecc.
filtered_reviews = only_text[
    only_text['review_text'].str.contains('spoiler', case=False) &  # contains "spoiler"
    ~only_text['review_text'].str.contains('no spoiler|not a spoiler|non spoiler|not to spoiler|not spoiler|any spoiler|[^not] spoilers* free', case=False)  # doesn't contain exclusion phrases
]


In [ ]:
spoilers = only_text[only_text['is_spoiler'] == True]
labels_3 = ['Only marked as spoiler','Contain also a textual spoiler alert',]
values_2 = [len(spoilers), len(filtered_reviews[filtered_reviews['is_spoiler'] == True])]
colors_2 = ['#ACBED8','#41608b']

plt.pie(values_2, labels=labels_3, colors=colors_2, autopct='%1.1f%%', startangle=140)

# Visualizzazione del plot
plt.axis('equal')
plt.show()


### More exploration

In [ ]:
#HOW MANY OF THE "ANNOUNCED SPOILERS" ARE ACTUALLY SPOILERS?
labels_2 = ['Are spoilers', 'Are not']
values_2 = [len(filtered_reviews[filtered_reviews['is_spoiler'] == True]),len(filtered_reviews[filtered_reviews['is_spoiler'] == False]) ]
colors_2 = ['#D52941', '#65B891']
plt.bar(labels_2, values_2, color=colors_2, width=0.9)

In [ ]:
#Number of reviews per user
user_total_comments = dataRew.groupby('user_id').size().reset_index(name='count_total')
user_total_comments.hist(bins=100, range=(0,8))

In [ ]:
user_total_comments.count_total.mean()

## Processing Text
In this part of the notebook, we perform the process of tokenization, removal of stop words, punctuation and lemmatization.

## Review Cleaning

Initialize pandarallel using all available CPU cores on the machine

In [ ]:
# inizialize pandarallel for work in parallel on the dataset

pandarallel.initialize(progress_bar=True)

In [ ]:
# this function is used to read the partial dataset, if we save dataset at each operation.
def takeDataset(x):
    ## When i read the csv, the last column is not an array but a string
    dataRew=pd.read_csv(x)
    ## Converting the string in an array before delete stop words
    dataRew["clean_review"]=dataRew.loc[:,"clean_review"].parallel_apply(ast.literal_eval)

### Tokenization

In [ ]:
nltk.download('punkt')

In [ ]:
def tokenize_text(text):
    from nltk.tokenize import WordPunctTokenizer
    tokenizer=WordPunctTokenizer()
    return tokenizer.tokenize(text)


In [ ]:
dataRew['clean_review'] = dataRew.loc[:,"review_text"].parallel_apply(tokenize_text)

### Delete StopWords

In [ ]:
nltk.download('stopwords')

In [ ]:
def remove_Stop(x):
  from nltk.corpus import stopwords
  stop_words = set(stopwords.words('english'))
  filtered_sentence = [word for word in x if word.lower() not in stop_words]
  return filtered_sentence

In [ ]:
dataRew['clean_review']=dataRew.loc[:,'clean_review'].parallel_apply(remove_Stop)

### Regex to  remove of punct.

In [ ]:
def remove_punct2(words_list):
    import re
    pattern=re.compile(r'[^\w\s]')
    clean_words=[word for word in words_list if not pattern.search(word)]
    return clean_words

In [ ]:
dataRew['clean_review']=dataRew.loc[:,'clean_review'].parallel_apply(remove_punct2)

In [ ]:
dataRew["clean_review"]

In [ ]:
dataRew.loc[:,"clean_review"]

### Lemmatization

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
def lemmatize(text):
    from nltk.stem import WordNetLemmatizer
    lemmatizer=WordNetLemmatizer()
    lem_token=[lemmatizer.lemmatize(token.lower()) for token in text]
    return lem_token


In [ ]:
dataRew['clean_review']=dataRew.loc[:,'clean_review'].parallel_apply(lemmatize)

In [ ]:
dataRew.loc[:,"clean_review"]

Let's save the cleaned dataset for later use

In [ ]:
dataRew.to_csv('../Dataset/datiClean.csv', index=False)

### Stemming
We also tried stemming in addition to lemmatization, but we opted for the latter since stemming appears too "aggressive".

In [ ]:
def stem_text(x):
  from nltk.stem import SnowballStemmer
  stemmer=SnowballStemmer('english')
  s_words=[stemmer.stem(word) for word in x]
  return s_words

In [ ]:
def ApplyStem():
    dataRew['clean_review']=dataRew.loc[:,'clean_review'].parallel_apply(stem_text)

## Processing synopsis of the plot
In this part we apply the same processes we made for text review

### Tokenize

In [ ]:
def tokenize_text(text):
   from nltk.tokenize import WordPunctTokenizer
   tokenizer=WordPunctTokenizer()
   return tokenizer.tokenize(text)

In [ ]:
dataMovie["plot_clean"]=dataMovie.loc[:,"plot_synopsis"].parallel_apply(tokenize_text)

In [ ]:
dataMovie["plot_clean"]

### Remove Stop Words

In [ ]:
nltk.download('stopwords')
def remove_Stop(x):
   from nltk.corpus import stopwords
   stop_words=set(stopwords.words('english'))
   filtered_sentence=[word for word in x if word.lower() not in stop_words]
   return filtered_sentence

In [ ]:
dataMovie["plot_clean"]=dataMovie.loc[:,"plot_clean"].parallel_apply(remove_Stop)

In [ ]:
dataMovie["plot_clean"]

### Remove punctuation

Let's use the same function used for the review text

In [ ]:
dataMovie['plot_clean']=dataMovie.loc[:,'plot_clean'].parallel_apply(remove_punct2)

In [ ]:
dataMovie["plot_clean"]

## Lemmatization

In [ ]:
def lemmatize(text):
    from nltk.stem import WordNetLemmatizer
    lemmatizer=WordNetLemmatizer()
    lem_token=[lemmatizer.lemmatize(token.lower()) for token in text]
    return lem_token


In [ ]:
dataMovie['plot_clean']=dataMovie.loc[:,'plot_clean'].parallel_apply(lemmatize)

### Stemming
Not used, we also tried stemming in addition to lemmatization, but we opted for the latter since stemming appears too "aggressive".

In [ ]:
def Stem(x):
   from nltk.stem import PorterStemmer
   stemmer=PorterStemmer()
   words=[stemmer.stem(word) for word in x]
   return words

### Save the Dataset

In [ ]:
dataMovie.to_csv("../Dataset/movieclean.csv")

In [ ]:
dataMovie